In [3]:
#Fill in the lines below to use this sheet to turn DOE designs or experimental designs into ECHO Sheets. 
#This will be the 
import pandas as pd
import numpy as np

#Add location of the excel sheet that will be used: 
experiment_sheet = '11_14_24_pyrogallol_melanin.xlsx'

#Choose what kind of plate you are using, change number to 25 if a 384 or 49 if a 1536 well plate:
plate_type = 49

#dispensing volumes used to create the echo sheet
experiment_grid = 'experiment_grid'

#add location of the volume matrix for the plate, this is a sheet called dispensing volume, and the condition sheet for each dispensing well. 
dispensing = 'dispensing_volume'
conditions = 'dispensing_conditions'

#location of the data that will be headered and used for downstream analysis: 
plate_reader_data = 'plate_reader_data'

In [4]:
import pandas as pd

# Read the data from the Excel sheet "experiment grid"
file_path = experiment_sheet  # Update this path to your Excel file
df = pd.read_excel(file_path, sheet_name='experiment_grid')

# Set the threshold (e.g., 5000)
threshold = 23000

# Initialize the new DataFrame with the Run column
new_df = pd.DataFrame(df['Run'])

# Function to add new columns dynamically
def add_new_column(new_df, col, suffix):
    new_col = f"{col}_{suffix}"
    new_df[new_col] = 0
    return new_col

# Split the columns and move values based on the threshold
for col in df.columns:
    if col != 'Run':
        suffix = 1
        new_col = add_new_column(new_df, col, suffix)
        total = 0
        for i, value in enumerate(df[col]):
            if pd.isna(value):  # Check for NaN (blanks) and treat them as zeros
                value = 0
            if total + value > threshold:
                suffix += 1
                new_col = add_new_column(new_df, col, suffix)
                total = 0
            new_df.at[i, new_col] = value
            total += value

# Write the new DataFrame to a new sheet in the same Excel file
with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace') as writer:
    new_df.to_excel(writer, sheet_name='dispensing_volume', index=False)

print("'dispensing_volume' has been added.")


'dispensing_volume' has been added.


In [7]:
import pandas as pd

# Function to generate 384-well pattern in row-major order
def generate_384_well_pattern(start_well):
    rows = 'ABCDEFGHIJKLMNOP'
    cols = range(1, 25)
    well_pattern = [f"{row}{col}" for row in rows for col in cols]
    
    if start_well not in well_pattern:
        raise ValueError("Invalid starting well position.")
    
    start_index = well_pattern.index(start_well)
    return well_pattern[start_index:] + well_pattern[:start_index]

# Function to get the Source Plate Type from the user
def get_source_plate_types(sample_ids, default_type, available_types):
    plate_types = [default_type] * len(sample_ids)
    exceptions = input("Enter Sample IDs and their Source Plate Types (format: ID1:Type1, ID2:Type2,...), or press Enter to skip: ")
    
    if exceptions:
        exception_list = exceptions.split(',')
        for exception in exception_list:
            sample_id, plate_type = exception.split(':')
            sample_id = sample_id.strip()
            plate_type = plate_type.strip()
            
            if sample_id in sample_ids and plate_type in available_types:
                index = sample_ids.index(sample_id)
                plate_types[index] = plate_type
            else:
                print(f"Invalid entry: {sample_id} or {plate_type} is not recognized.")
    
    return plate_types

# Read the data from the Excel sheet "dispensing_volume"
file_path = experiment_sheet  # Update this path to your Excel file
df_dispensing_volume = pd.read_excel(file_path, sheet_name='dispensing_volume')

# Extract the headers excluding the first one and transpose them into a new DataFrame
dispensing_conditions = pd.DataFrame(df_dispensing_volume.columns[1:], columns=['Sample ID'])

# Ask user for the starting well
start_well = input("Enter the starting well (e.g., B1): ")

# Generate the 384-well pattern starting from the specified well
well_pattern = generate_384_well_pattern(start_well)

# Fill the 'Source Well' column with the generated pattern
dispensing_conditions['Source Well'] = well_pattern[:len(dispensing_conditions)]

# Define available Source Plate Types
available_types = [
    '384PP_Plus_AQ', '384PP_AQ_BP', '384PP_AQ_SP', '384PP_AQ_SP_High',
    '384PP_Plus_AQ_BP', '384PP_Plus_AQ_GP', '384PP_Plus_AQ_GPSA',
    '384PP_Plus_AQ_GPSB', '384PP_Plus_AQ_SP'
]

# Get the Source Plate Type from the user
default_type = '384PP_Plus_AQ'
sample_ids = dispensing_conditions['Sample ID'].tolist()
source_plate_types = get_source_plate_types(sample_ids, default_type, available_types)

# Fill the 'Source Plate Type' column with the appropriate values
dispensing_conditions['Source Plate Type'] = source_plate_types

# Print the resulting DataFrame for testing
print(dispensing_conditions)

# Write the new DataFrame to a new sheet in the same Excel file
with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace') as writer:
    dispensing_conditions.to_excel(writer, sheet_name='dispensing_conditions', index=False)

print("The new sheet 'dispensing_conditions' has been added to the Excel file.")


Enter the starting well (e.g., B1):  A1
Enter Sample IDs and their Source Plate Types (format: ID1:Type1, ID2:Type2,...), or press Enter to skip:  


                       Sample ID Source Well Source Plate Type
0   Laccase A. niger(0.2mg/mL)_1          A1     384PP_Plus_AQ
1     Laccase A. niger(2mg/mL)_1          A2     384PP_Plus_AQ
2      1_3 dihydroxynapthalene_1          A3     384PP_Plus_AQ
3      2_7 dihydroxynapthalene_1          A4     384PP_Plus_AQ
4      1_4 dihydroxynapthalene_1          A5     384PP_Plus_AQ
5                  Pyrogallol _1          A6     384PP_Plus_AQ
6                  Pyrogallol _2          A7     384PP_Plus_AQ
7      1_5 dihydroxynapthalene_1          A8     384PP_Plus_AQ
8                     Catechol_1          A9     384PP_Plus_AQ
9      1_6 dihydroxynapthalene_1         A10     384PP_Plus_AQ
10  3_4-dihydroxybenzoic acid _1         A11     384PP_Plus_AQ
11     1_7 dihydroxynapthalene_1         A12     384PP_Plus_AQ
12      4-hydroxybenzoic acid _1         A13     384PP_Plus_AQ
13     1_8 dihydroxynapthalene_1         A14     384PP_Plus_AQ
14      3-hydroxybenzoic acid _1         A15     384PP_

In [8]:
import pandas as pd

def well_to_index(start_well):
    # Split the input into row (letter) and column (number) parts
    row = start_well[:-1].upper()  # Row part (e.g., "A" from "A1")
    col = int(start_well[-1:])  # Column part (e.g., "1" from "A1")

    # List of rows from A to Z and double letters for larger plates
    rows = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ') + [chr(i) + chr(i) for i in range(65, 91)]
    
    # Calculate the index based on the row and column
    row_index = rows.index(row)
    index = row_index * plate_type + (col - 1)  # Adjust to 0-based index
    return index

def run_to_well(run, start_well='A1'):
    # List of rows from A to Z and double letters for larger plates
    rows = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ') + [chr(i) + chr(i) for i in range(65, 91)]
    # List of columns based on the plate type
    cols = list(range(1, plate_type + 1))
    # Creating the well labels
    wells = [f'{r}{c}' for r in rows for c in cols]
    
    # Get the start well index
    start_index = well_to_index(start_well)
    
    # Adjust the run by the start_well index
    return wells[run + start_index - 1]  # Adjusted for 0-based indexing

# Ask the user for the starting well
start_well = input("Please enter the starting well (e.g., A1): ")

# Read the first sheet of the Excel file into a DataFrame
df = pd.read_excel(experiment_sheet, sheet_name=dispensing)

# Transpose the DataFrame so that the components are columns
df = df.transpose()

# Reset the index so that the components become a column
df.reset_index(inplace=True)

# Rename the columns to reflect the new structure
df.columns = df.iloc[0]
df = df[1:]

# Rename the 'Run' column to 'Sample ID'
df.rename(columns={'Run': 'Sample ID'}, inplace=True)

# Read the second sheet (with the 'Source Well', 'Source Plate Type', 
# and offsets for each well) into a DataFrame
df_source_data = pd.read_excel(experiment_sheet, sheet_name=conditions)

# Melt the DataFrame from wide format to long format
df_melted = df.melt(id_vars='Sample ID', var_name='Run', value_name='Transfer Volume')

# Convert 'Run' to numeric
df_melted['Run'] = pd.to_numeric(df_melted['Run'])

# Apply run_to_well function to 'Run' column to get 'Destination Well', starting from the specified well
df_melted['Destination Well'] = df_melted['Run'].apply(lambda x: run_to_well(x, start_well=start_well))

# Merge df_melted with df_source_data to get corresponding Source Well, Source Plate Type, and X/Y Offsets for each Sample ID
df_melted = pd.merge(df_melted, df_source_data, on='Sample ID', how='left')

# Add the additional columns
df_melted['Source Plate'] = 'Source[1]'
df_melted['Destination Plate Name'] = '384PP_Dest'

# Ensure the DataFrame includes 'Destination Well X Offset' and 'Destination Well Y Offset' from the source data
if 'Destination Well X Offset' not in df_melted.columns:
    df_melted['Destination Well X Offset'] = 0  # default if not provided
if 'Destination Well Y Offset' not in df_melted.columns:
    df_melted['Destination Well Y Offset'] = 0  # default if not provided

# Reorder the columns to match the desired output
df_melted = df_melted[['Source Plate', 'Source Plate Type', 'Source Well', 'Sample ID', 
                       'Destination Plate Name', 'Destination Well', 'Transfer Volume',
                       'Destination Well X Offset', 'Destination Well Y Offset']]

# Specify the output file path
output_file_path = experiment_sheet

# Specify the name of the new sheet
new_sheet_name = 'ECHO_SHEET'

# Write the DataFrame to a new sheet in the same Excel file
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a') as writer:
    df_melted.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(df_melted)


Please enter the starting well (e.g., A1):  A1


     Source Plate Source Plate Type Source Well                     Sample ID  \
0       Source[1]  384PP_Plus_AQ_BP          F1  Laccase A. niger(0.2mg/mL)_1   
1       Source[1]  384PP_Plus_AQ_BP          F2    Laccase A. niger(2mg/mL)_1   
2       Source[1]  384PP_Plus_AQ_BP          F3     1_3 dihydroxynapthalene_1   
3       Source[1]  384PP_Plus_AQ_BP          F4     2_7 dihydroxynapthalene_1   
4       Source[1]  384PP_Plus_AQ_BP          F5     1_4 dihydroxynapthalene_1   
...           ...               ...         ...                           ...   
2195    Source[1]  384PP_Plus_AQ_BP         F21                      Phenol_1   
2196    Source[1]  384PP_Plus_AQ_BP         F22                   1-napthol_1   
2197    Source[1]  384PP_Plus_AQ_BP         F23                   2-napthol_1   
2198    Source[1]  384PP_Plus_AQ_BP         F24                  Anthracene_1   
2199    Source[1]  384PP_Plus_AQ_BP          G1               2-Aminophenol_1   

     Destination Plate Name

In [3]:
import openpyxl

# Define the path to your Excel file
file_path = experiment_sheet  # Update this path to your Excel file

# Open the workbook and select the sheet
wb = openpyxl.load_workbook(file_path)
experiment_grid = 'experiment_grid'
plate_reader_data = 'plate_reader_data'

# Read reagents from the first row, skipping the first cell
sheet = wb[experiment_grid]
reagents = [cell.value for cell in sheet[1]][1:]

# Read data from the sheet
data = {}
for row in sheet.iter_rows(min_row=2, values_only=True):  # Skip the first row (headers)
    test = row[0]
    values = list(row[1:])  # Skip the first column (test names)
    data[test] = values

# Create headers
headers = {}
for test, values in data.items():
    headers[test] = []
    for i, value in enumerate(values):
        if value != 0 and value is not None:  # Check for non-zero and non-None values
            headers[test].append(f"{reagents[i]}:{value}")

# Convert list of headers to a single string for each test
output = []
for test, values in headers.items():
    if values:  # Only add to output if there are headers
        headers_string = ', '.join(values)
    else:
        headers_string = 'None'  # Add 'None' if there are no headers
    output.append(headers_string)  # Removed test name from output

# Create or open the 'plate_reader_data' sheet
if plate_reader_data in wb.sheetnames:
    sheet = wb[plate_reader_data]
else:
    sheet = wb.create_sheet(title=plate_reader_data)

# Clear existing content in the 'plate_reader_data' sheet
for row in sheet.iter_rows():
    for cell in row:
        cell.value = None

# Write output to the top row of the 'plate_reader_data' sheet in an Excel file
for i, header in enumerate(output, start=2):  # Start from the second column to avoid the time column
    sheet.cell(row=1, column=i, value=header)

# Save the workbook
wb.save(file_path)

print("The new sheet 'plate_reader_data' has been added to the Excel file.")


The new sheet 'plate_reader_data' has been added to the Excel file.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the file path
data = experiment_sheet
sheet_name= 'plate_reader_data'

# Read the excel file and specify the sheet name
df = pd.read_excel(data, sheet_name)

# Convert the dataframe to a numpy array
df = df.to_numpy()

# Ignore the first column and use it as time
time = df[:, 0]

# Define a function to take the average or standard deviation of every x columns
def agg_columns(df, x, agg_func):
    # Create an empty list to store the results
    results = []
    # Loop through the columns in groups of x
    for i in range(1, len(df[0]), x):
        # Calculate the aggregation of the group and append it to the list
        results.append(agg_func(df[:, i:i+x], axis=1))
    # Return the list of results as a new dataframe
    return pd.DataFrame(results).T

# Set x and y to the desired values
x = 4
y = 24

# Get the headers from the original dataframe
headers = pd.read_excel(data, sheet_name).columns

# Calculate number of rows and columns for subplots
nrows = int(np.ceil(8))
ncols = min(1, y)

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(40, 100), dpi=300)
axs = axs.flatten()  # to handle case when y < 4

# Loop through the data by y columns
for j in range(0, len(df[0]), y):
    # Call the agg_columns function and get the averages and standard deviations dataframes for current y columns
    averages = agg_columns(df[:, j:j+y], x, np.mean)
    stds = agg_columns(df[:, j:j+y], x, np.std)

    # Set the font family and size
    plt.rc('font', family='Arial', size=24)

    # Loop through the columns of the averages dataframe
    for i in range(len(averages.columns)):
        # Plot the column as a line against time with a color and marker
        axs[j//y].plot(time, averages.iloc[:, i], label=str(headers[i*x+j+1]), color="C"+str(i), marker="o")
        # Calculate the upper and lower bounds of the shadow
        upper = averages.iloc[:, i] + stds.iloc[:, i]
        lower = averages.iloc[:, i] - stds.iloc[:, i]
        # Fill the area between the bounds with a transparent color
        axs[j//y].fill_between(time, upper, lower, color="C"+str(i), alpha=0.2)

    # Add labels and title with descriptive text
    axs[j//y].set_xlabel("Time (Mins)")
    axs[j//y].set_ylabel("RFU")
    axs[j//y].set_title("")
    
    #set the Y-Lim
    axs[j//y].set_ylim([0, 12000])
    
    


    # Add a legend with a location and frame
    axs[j//y].legend(loc="upper left", frameon=True, fontsize=30)
    

    # Add grid lines for readability
    #axs[j//y].grid(True, linestyle='--', color='gray', linewidth=0.5)

    # Set axis spines to be invisible except bottom one
    axs[j//y].spines['top'].set_visible(False)
    axs[j//y].spines['right'].set_visible(False)
    
# Save each figure as a high-quality PNG image with unique name based on j value
plt.savefig("plot.png", format="png", dpi=300)

# Show each plot separately on screen 
plt.show()
